## Динамические алгоритмы и внешнее обучение

Внешнее обучение - подход, который даёт возможность работать с крупными наборами данных, постепенно подгоняя классификатор на меньших пакетах из набора данных.

Функция partial_fit (SGDClassifier) используется для потоковой передачи документов прямо из локалього диска.

Функция tokenizer очищает необработанные текстовые данные из файла и разбивает его на лексемы, одновременно удаляя стоп-слова:

In [1]:
import numpy as np
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(<?::|;|=) (?:-)?(?:\)|\(|D|P)',
                           text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
                  + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

Генераторная функция stream_docs - читает и возвращает один документ за раз:

In [44]:
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # пропустить заголовок
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label  # возвращение генератора

In [45]:
next(stream_docs(path='movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

Функция get_minibatch принимает поток документов от функции stream_docs и возвращает определенное число документов, указанное в параметре:

In [46]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

Класс CountVectorizer требует удержания полного глоссария в памяти. Классу TfidVectorizer необходимо хранить в памяти все векторы признаков обучающего набора, чтобы вычислять обратные частоты документов.

HashingVectorizer не зависит от данных и задействует трюк с хешированием через 32-битную функцию MurmurHash3:

In [47]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)
clf = SGDClassifier(loss='log', random_state=1)
doc_stream = stream_docs(path='movie_data.csv')

Внешнее обучение с выводом прогрессбара:

Проходим по 45 минипакетам документов, где каждый минипакет состоит из 1000 документов.

In [48]:
import pyprind
pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])

for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:21


Применяем последние 5000 документов для оценки эффективности модели:

In [49]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('acc: ', clf.score(X_test, y_test))

acc:  0.8682


In [50]:
clf = clf.partial_fit(X_test, y_test)

acc:  0.8838


Более современной альтернативой модели суммирования слов является word2vec.

Алгоритм word2vec - это алгоритм обучения без учителя, основанный на нейронных сетях, который пытается автоматически узнать
взаимосвязь между словами. Идея, лежащая в основе word2vec, заключается в том, чтобы помещать слова с похожим смыслом в подобные кластеры, и посредством искусной организации пространства векторов модель способна воспроизводить определенные слова с использованием простой векторной математики, например, king - man + wоman = queen

## Тематическое моделирование с помощью латентного размещения Дирихле

Тематическое моделирование - задача кластеризации.

Latent Dirichlet Allocation (LatDA) - алгоритм без учителя.
LatDA - порождающая вероятностная модель, которая пытается отыскать группы слов, часто появляющихся вместе в различных документах.

На вход LatDA получает модель суммирования слов и разлагает ее на 2 новые матрицы:
- матрица отображения документов на темы
- матрица отображения слов на темы

LatDA разлагает матрицу суммирования слов на 2 матрицы таким образом, что если мы перемножим эти две матрицы, то будем в состоянии воспроизвести вход, т.е. матрицу суммирования слов, с самой низкой возможной ошибкой. На практике нас интересуют темы, которые прием LatDA нашел в матрице суммирования слов.
Количество тем должно быть определено заранием (является гиперпараметром LatDA)

In [52]:
import pandas as pd
df = pd.read_csv('movie_data.csv')

In [53]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(stop_words='english',
                        max_df=.1,
                        max_features=5000)
X = count.fit_transform(df['review'].values)

Максимальная частота слов, подлежащая расмотрению равна 10% (max_df=0.1), чтобы исключить слова, которые встречаются в документах слишком часто.

Также ограничивается количество учитываемых слов пятью тысячами чаще всего встречающихся слов (max_features=5000) для установлениия лимита на размерность набора данных, чтобы улучшить выведение, выполняемое LatDA.

In [55]:
from sklearn.decomposition import LatentDirichletAllocation

latda = LatentDirichletAllocation(n_components=10,
                                  random_state=123,
                                  learning_method='batch',
                                  n_jobs=-1)
X_topics = latda.fit_transform(X)

Матрица, содержащая значения важности слов для каждой из 10 тем в порядке возрастания:

In [58]:
latda.components_.shape

(10, 5000)

Вывод пяти самых важных слов для каждой из 10 тем:

In [61]:
n_top_words = 5
feature_names = count.get_feature_names_out()
for topic_i, topic in enumerate(latda.components_):
    print(f'Тема: {topic_i + 1}')
    print(' '.join([feature_names[i] for i in topic.argsort()\
                    [:-n_top_words - 1:-1]]))

Тема: 1
worst minutes script awful stupid
Тема: 2
family mother father children girl
Тема: 3
american dvd music tv war
Тема: 4
human audience cinema art feel
Тема: 5
police guy car dead murder
Тема: 6
horror house sex woman girl
Тема: 7
role performance comedy actor performances
Тема: 8
series episode war episodes season
Тема: 9
book version original effects special
Тема: 10
action fight guy guys fun


Предроложение о темах:
1) Плохие фильмы (не являются по-настоящему тематичской категорией)
2) Семейные фильмы
3) Военные фильмы
4) Фильмы об искусстве
5) Криминальнные фильмы
6) Фильмы ужасов
7) Комедийные фильмы
8) Фильмы, связанные с телевизионными шоу
9) Фильмы по мотивам книг
10) Фильмы боевики

In [63]:
horror = X_topics[:, 5].argsort()[::-1]
for i, movie_i in enumerate(horror[:3]):
    print(f'horror movie: {i + 1}')
    print(df['review'][movie_i][:300], '...')

horror movie: 1
Once upon a time in a castle...... Two little girls are playing in the garden's castle. They are sisters. A blonde little girl (Kitty) and a brunette one (Evelyn). Evelyn steals Kitty's doll. Kitty pursues Evelyn. Running through long corridors, they reach the room where their grandfather, sitting o ...
horror movie: 2
House of Dracula works from the same basic premise as House of Frankenstein from the year before; namely that Universal's three most famous monsters; Dracula, Frankenstein's Monster and The Wolf Man are appearing in the movie together. Naturally, the film is rather messy therefore, but the fact that ...
horror movie: 3
<br /><br />Horror movie time, Japanese style. Uzumaki/Spiral was a total freakfest from start to finish. A fun freakfest at that, but at times it was a tad too reliant on kitsch rather than the horror. The story is difficult to summarize succinctly: a carefree, normal teenage girl starts coming fac ...


In [ ]:
341